In [ ]:
# This code is from https://platform.openai.com/docs/tutorials/meeting-minutes
# This code generates a word document for minutes of a meeting from the audio transcription of the meeting
# It uses OpenAI's Whisper API and GPT-4 Model. You need to have a paid account and valid OPENAI API Key
# First install these two required libraries
pip install openai
pip install python-docx

In [ ]:
import openai
from docx import Document
# audio files in following formats work with OpenAI's Audio Module : mp3, mp4, mpeg, mpga, m4a, wav, or webm
audio_file_path = "meeting_audio.wav"
audio_file_object = open(audio_file_path, 'rb')
transcription = openai.Audio.transcribe("whisper-1", audio_file)
transcription = transcription['text']

#Get summary of transcription from GPT-4 using prompting
response = openai.ChatCompletion.create(model="gpt-4",temperature=0,
        messages=[
            {  
                "role": "system",
                "content": "Please summarize the following text into a coherent and readable paragraph. \
                Avoid unnecessary details."
            },
            {   "role": "user", 
                "content": transcription 
            }   ]   )
abstract_summary = response['choices'][0]['message']['content']

#Get keypoints of the transcription from GPT-4 using prompting
response = openai.ChatCompletion.create(model="gpt-4",temperature=0,
        messages=[
            {   "role": "system",
                "content": "From the following text, identify and list the important ideas, \
                topics, findings that were discussed"
            },
            {   "role": "user",
                "content": transcription 
            }  ]    )
key_points = response['choices'][0]['message']['content']

#Extract action items from transcription using GPT-4 prompting
response = openai.ChatCompletion.create(model="gpt-4",temperature=0,
        messages=[
            {   "role": "system",
                "content": "You are an AI expert in analyzing conversations and extracting action items.\
                Review the text and identify any tasks, assignments, or actions that were agreed upon \
                or mentioned as needing to be done. These could be tasks assigned to specific individuals, or \
                general actions that the group has decided to take. Please list these action items clearly and concisely."
            },   
            {   "role": "user",
                "content": transcription 
            }  ]  )
action_items = response['choices'][0]['message']['content']

#Get sentiments present in the transcription using GPT-4
response = openai.ChatCompletion.create(model="gpt-4",temperature=0,
        messages=[
            {
                "role": "system",
                "content": "As an AI with expertise in language and emotion analysis, your task is to analyze \
                the sentiment of the following text. Please consider the overall tone of the discussion, \
                the emotion conveyed by the language used, and the context in which words and phrases are used.\
                Indicate whether the sentiment is generally positive, negative, or neutral, and provide brief\
                explanations for your analysis where possible."
            },  
            {  "role": "user",    
             "content": transcription   
            }  ]   )
sentiment = response['choices'][0]['message']['content']

#Combining summary, keypoints, actionitems and sentiment in a single dictionary
final_results = {'abstract_summary': abstract_summary,'key_points': key_points,'action_items': action_items,'sentiment': sentiment}

# Save the results to a docx file
filename = "MoM.docx"
doc = Document()
for key, value in final_results.items():
    # Replace underscores with spaces and capitalize each word for the heading
    heading = ' '.join(word.capitalize() for word in key.split('_'))
    doc.add_heading(heading, level=1)
    doc.add_paragraph(value)
    # Add a line break between sections
    doc.add_paragraph()
doc.save(filename)